In [ ]:
!python -m pip install seaborn
!python -m pip install scikit-learn

In [ ]:
!python -m pip install "elasticsearch>=6.4.0,<7.0.0"

In [ ]:
from elasticsearch import Elasticsearch
client = Elasticsearch("http://172.17.212.35:9200")
resp = client.search(index="retromotion-indexer_development_products",
                     body={"_source":["descriptions","descriptionsSource","nameSource","shortDescriptionSource","categoriesSource"],
                           "query": {"match_all": {}}})


In [ ]:
print("Got %d Hits:" % resp['hits']['total'])

In [ ]:
import pandas as pd
import re
import numpy as np

import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_eng = pd.DataFrame(columns=['name','sdesc','category','catlevel0','catlevel1','catlevel2','catlevel3','catlevel4','catlevel5'])
df_de = pd.DataFrame(columns=['name','sdesc','desc','category','categorypath'])

In [18]:
df_eng.iloc[0]

name         denso dtm82363 thermostat coolant
sdesc                       thermostat coolant
category                            thermostat
catlevel0                            sparepart
catlevel1                       cooling system
catlevel2                           thermostat
catlevel3                                  NaN
catlevel4                                  NaN
catlevel5                                  NaN
Name: 0, dtype: object

In [ ]:
def cleanStr(text):
   
    # remove numbers
    # no_number_string = re.sub(r'\d+','',lower_string)

    # remove all punctuation except words and space
    # text = re.sub(r'[^\w\s]','', lower_string)

    # clean = re.compile('<.*?>')

    # text = re.sub(clean,'',text)
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' .join(emoticons).replace('-', ''))

    return text


In [ ]:
def readHtml(html):
    dftable = pd.read_html(html)
    return dftable

In [ ]:
name_en=[]
name_de=[]

sdesc_en=[]
sdesc_de=[]

desc_en=[]
desc_de=[]

cat_en=[]
cat_de=[]

catpath_en=[]
catpath_de=[]

for hit in resp['hits']['hits']:
    list_row =[]
    for name in hit['_source']['nameSource']:
        if (name["language"]=="en"):
            list_row.append(cleanStr(name["value"]))
        else:
            name_de.append(name["value"])
    
    for sdesc in hit['_source']['shortDescriptionSource']:
        if (sdesc["language"]=="en"):
            list_row.append(cleanStr(sdesc["value"]))
        else:
            sdesc_de.append(sdesc["value"])

    # for desc in hit['_source']['descriptionsSource']:
    #     if (desc["language"]=="en"):
    #         desc_en.append(readHtml(desc["value"]))
    #     else:
    #         desc_de.append(desc["value"])
    
    for cats in hit['_source']['categoriesSource']:
        if (cats["language"]=="en"):
            list_row.append(cleanStr(cats["label"]))
        else:
            desc_de.append(cats["label"])

        if (cats["language"]=="en"):
            categories =cats["path"].split('/')
            i =0
            catls=[]
            for category in categories:

                list_row.append(cleanStr(category))
                i= i+1
            
        else:
            catpath_de.append(cats["path"])
    
    # new_row = pd.DataFrame({'name':_name, 'sdesc':_sdesc, 'category':_cat_en }, index=[0])

    # pd.concat([new_row,df_eng.loc[:]]).reset_index(drop=True)

    for i in range (0,9-len(list_row)):
        list_row.append(np.nan)

    df_eng.loc[len(df_eng)] = list_row



In [ ]:
# df_en = pd.DataFrame({'name':name_en,
#                        'sdesc':sdesc_en,
#                     #    'desc':desc_en,
#                        'category':cat_en,
#                        'categorypath':catpath_en})


# df_de = pd.DataFrame({'name':name_de,
#                        'sdesc':sdesc_de,
#                        'desc':desc_de,
#                        'category':cat_de,
#                        'categorypath':catpath_de})

In [ ]:
df_eng[0]

In [ ]:
print (df_eng.to_latex())

In [ ]:
df_eng.isna().sum()

In [ ]:
df_eng.shape

In [ ]:
df_eng = df_eng.dropna()

In [ ]:
df_eng.shape

In [ ]:
df_eng.head()

In [ ]:
df_eng.describe()

One hot encoding 

In [ ]:
# df_eng = pd.get_dummies(df_eng,columns=["category"])
# df_eng.head()

In [59]:
df_eng.iloc[7]

name         vaico v10 2660 mounting automatic transmission
sdesc                       mounting automatic transmission
category                                           mounting
catlevel0                                         sparepart
catlevel1                                      transmission
catlevel2                            automatic transmission
catlevel3                                          mounting
catlevel4                                               NaN
catlevel5                                               NaN
Name: 7, dtype: object

In [66]:
from sklearn.feature_extraction.text import CountVectorizer
n_gram_vectorizer = CountVectorizer(ngram_range=(1, 1))
transformed_vector_name = n_gram_vectorizer.fit_transform(df_eng['name'])
transformed_vector_sdesc = n_gram_vectorizer.fit_transform(df_eng['sdesc'])
transformed_vector_category = n_gram_vectorizer.fit_transform(df_eng['category'])

# vocabulary


In [72]:
transformed_vector_name

<10x40 sparse matrix of type '<class 'numpy.int64'>'
	with 54 stored elements in Compressed Sparse Row format>

In [68]:
n_gram_vectorizer.vocabulary_.get('automatic transmission')

In [69]:
vocabulary.items()

dict_items([('thermostat', 14), ('coolant', 6), ('automatic', 2), ('transmission', 17), ('camshaft', 4), ('track', 16), ('control', 5), ('arm', 1), ('suspension', 12), ('kit', 8), ('tensioner', 13), ('pulley', 10), ('timing', 15), ('belt', 3), ('mounting', 9), ('shock', 11), ('absorber', 0), ('wheel', 18), ('hub', 7)])

In [70]:
from sklearn.model_selection import train_test_split

X_train, x_test, Y_train, y_test = train_test_split(transformed_vector_sdesc,
                                                    transformed_vector_category,
                                                    test_size=0.2,
                                                    random_state=0)

In [71]:
X_train.shape, Y_train.shape

((8, 19), (8, 16))

In [4]:
import torch
# import numpy as np
# Xtrain_ = torch.from_numpy(X_train.values).float()
# Xtest_ = torch.from_numpy(x_test.values).float()
# Xtrain_.shape
# Ytrain_ = torch.from_numpy(Y_train.values).view(1,-1)[0]
# Ytest_ = torch.from_numpy(y_test.values).view(1,-1)[0]
# Ytrain_.shape

ModuleNotFoundError: No module named 'torch'

In [ ]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, hidden_size) 
        self.fc3 = nn.Linear(hidden_size, output_size) 
    
    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x)) 
        x = self.fc3(x)
        
        return F.log_softmax(x, dim=-1)

In [ ]:
model = Net()

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

loss_fn = nn.NLLLoss()

In [ ]:
epoch_data = []
epochs = 1001

for epoch in range(1, epochs):

    optimizer.zero_grad()
    Ypred = model(Xtrain_)

    loss = loss_fn(Ypred , Ytrain_)
    loss.backward()

    optimizer.step()
        
    Ypred_test = model(Xtest_)
    loss_test = loss_fn(Ypred_test, Ytest_)
    
    _,pred = Ypred_test.data.max(1)
    
    accuracy = pred.eq(Ytest_.data).sum().item() / y_test.values.size
    epoch_data.append([epoch, loss.data.item(), loss_test.data.item(), accuracy])
    
    if epoch % 100 == 0:
        print ('epoch - %d (%d%%) train loss - %.2f test loss - %.2f accuracy - %.4f'\
               % (epoch, epoch/150 * 10 , loss.data.item(), loss_test.data.item(), accuracy))